In [4]:
import sys
sys.path.append('NEW')

In [6]:
# Python
import numpy as np
import pandas as pd
import time

# Machine Learning
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# RDKit
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

# Our module
#from Python_Scoring_Export import Scoring, Export
#from Python_MLModel import RF, Ridge_M, XGB, NN
#from Python_RemoveO import remove_outliers


old_df = pd.DataFrame({
    'MAE':[0], 'MAPE(%)':[0], 'RMSE':[0], 'R2':[0], 'Radius':[0], 'nBits':[0], 'Model':[0]
    })

# =============================================================================
# # %% Option
# MF_bit = 2**5
# MF_radius = 6
# Name_model = "CB"
# =============================================================================

# %% Option Many Bit
MF_bit_s = [2**10]
MF_radius_s = [3]

#MF_bit_s = [2**5-1, 2**6-1]
#MF_radius_s = [3]
Name_model = "RF"
j=0
for MF_radius in MF_radius_s:
    for MF_bit in MF_bit_s :
        
        # %% 
        # Import Data
        df = pd.read_excel("../Data.xlsx",sheet_name="560point")
        #df = remove_outliers("Data.xlsx", "New_Data", 2)

        # Select feature for data: X=SMILE, Y=Tb
        X_data_excel= df[["SMILES"]]
        Y_data= df["Tb"]
        
        # %% Data Preparation
        # Generate Fingerprint from SMILE
        
        X_data_use = X_data_excel.copy()
        X_data_use["molecule"] = X_data_use["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
        X_data_use["morgan_fp"] = X_data_use["molecule"].apply(lambda x: rdMolDescriptors.GetMorganFingerprintAsBitVect(
                x, 
                radius=MF_radius, 
                nBits=MF_bit, 
                useFeatures=True, useChirality=True))
        
        # Transfrom Fingerprint to Column in DataFrame
        X_data_fp = []
        for i in range(X_data_use.shape[0]):
            #print(np.array(X_data_use["morgan_fp"][i]))
            array = np.array(X_data_use["morgan_fp"][i])
            datafram_i = pd.DataFrame(array)
            datafram_i = datafram_i.T
            X_data_fp.append(datafram_i)
        x_data_fp = pd.concat(X_data_fp, ignore_index=True)
        
        y_data_fp = Y_data.copy()

In [7]:
test = x_data_fp.ne(0).sum(axis=0)
top_bit = pd.DataFrame(test.sort_values(ascending=False)).head(50).index.to_numpy()
#top_bit

In [8]:
pd.DataFrame(test.sort_values(ascending=False)).head()

,0
0,559
61,478
792,463
644,275
464,179


In [9]:
def getSMART(mol, radius, atomidx):
    env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atomidx)
    atomsToUse = set((atomidx, ))
    for b in env:
        atomsToUse.add(mol.GetBondWithIdx(b).GetBeginAtomIdx())
        atomsToUse.add(mol.GetBondWithIdx(b).GetEndAtomIdx())
    enlargedEnv = set()
    for atom in atomsToUse:
        a = mol.GetAtomWithIdx(atom)
        for b in a.GetBonds():
            bidx = b.GetIdx()
            if bidx not in env:
                enlargedEnv.add(bidx)
    enlargedEnv = list(enlargedEnv)
    enlargedEnv += env
    # find all relevant neighbors
    anyAtoms = []
    for a in atomsToUse:
        neighbors = mol.GetAtomWithIdx(a).GetNeighbors()
        for n in neighbors:
            anyIdx = n.GetIdx()
            if anyIdx not in atomsToUse:
                anyAtoms.append(anyIdx)
    # replace atomic number to zero (there is no number for any atom)
    for aA in anyAtoms:
        mol.GetAtomWithIdx(aA).SetAtomicNum(0)
    submol = Chem.PathToSubmol(mol, enlargedEnv)
    # change [0] to *
    MorganBitSmarts = Chem.MolToSmarts(submol).replace('[#0]', '*')
    #print(MorganBitSmarts)
    return MorganBitSmarts 

def get_All_SMART_1_mol(mol, radius, nBits):
    bit_info = {}
    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(
                mol, 
                radius=radius, 
                nBits=nBits, 
                bitInfo = bit_info,
                useFeatures=True, useChirality=True)

    #print(get_index_of_array_that_contain_1_in_any_position(list(fp)))
    #print(bit_info.keys())
    string_SMARTS = dict()
    for i,v in enumerate(bit_info.keys()):
        #print(i)
        #print("v=",v)
        atomidx, radius = bit_info[v][0]
        string_SMART = getSMART(mol=mol, radius=radius, atomidx=atomidx)
        #string_SMART = "TEST"
        #print(v, string_SMART)
        string_SMARTS[v] = string_SMART
        #print(v)
    #print(string_SMARTS)
    #print(string_SMARTS)
    try :
        return string_SMARTS
        #print("HERE")
    except :
        None

In [10]:
text_SMILES = X_data_use["molecule"].iloc[1]
text_SMILES
get_All_SMART_1_mol(text_SMILES, 3, 1024)

{0: '[#6]-*', 61: '[#6]-*'}

In [11]:
radius = 3; nBits = 1024
X_data_use["All_SMART"] = X_data_use["molecule"].apply(lambda x : get_All_SMART_1_mol(x, radius, nBits))


In [12]:
X_data_use

,SMILES,molecule,morgan_fp,All_SMART
0,C,<rdkit.Chem.rdchem.Mol object at 0x000001454EF...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",{0: ''}
1,CC,<rdkit.Chem.rdchem.Mol object at 0x000001454EF...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: '[#6]-*', 61: '[#6]-*'}"
2,CCC,<rdkit.Chem.rdchem.Mol object at 0x000001454EF...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: '[#6]-*', 61: '[#6]-*-*', 792: '[#6]-*-*'}"
3,C1CC1,<rdkit.Chem.rdchem.Mol object at 0x000001454EF...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: '[#6](-*)-*', 464: '[#6]1-*-*-1', 792: '[#..."
4,CC(C)C,<rdkit.Chem.rdchem.Mol object at 0x000001454EF...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: '[#6]-*', 61: '[#6]-*(-*)-*', 644: '[#6]-*..."
...,...,...,...,...
555,CCCCCCC#CCCC,<rdkit.Chem.rdchem.Mol object at 0x000001454F5...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: '[#6]-*', 18: '*-*-[#6]-[#6]-[#6]-[#6]-[#6..."
556,CCCCCCCCCC,<rdkit.Chem.rdchem.Mol object at 0x000001454F5...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: '[#6]-*', 61: '[#6]-*-*', 252: '[#6]-*-*-[..."
557,CCCCCCCCC#CC,<rdkit.Chem.rdchem.Mol object at 0x000001454F5...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: '[#6]-*', 18: '*-[#6]-[#6]-[#6]-[#6]-[#6]-..."
558,CCCCCCCC1CCCC1,<rdkit.Chem.rdchem.Mol object at 0x000001454F5...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: '[#6]-*', 20: '*-[#6]-[#6]-[#6]-[#6]-[#6]-..."


In [ ]:
def cb(x, bit):
    try:
        return x[bit]
    except :
        None

for bit in top_bit:
    #print(bit)
    X_data_use[f"Bit {bit}"]  =  X_data_use["All_SMART"].apply(lambda x : cb(x,bit))

In [3]:
#X_data_use.to_excel("Check_bit_12k_top50.xlsx")

NameError: name 'X_data_use' is not defined

In [ ]:
top_bit